# Kluge2017 - Full Pipeline

This challenge tests the performance of a full gaitanalysis pipeline on laboratory 4x10m gait tests.
Comparisons are performed on the calculated means (TODO: and the variance) of spatial temporal parameters over the entire gait test.

Reference gait parameters are provided by a marker-less motion capture system that covers the center of the walking section.
As the MoCap system does not cover the turns at the end of each 10m section algorithms should also cut of all turning strides (recommendation: turning angle > 20 deg) before calculating the mean and the variance.

The entire validation is run as a 5-fold cross-validation to allow the algorithms to optimize parameters on an independent train set.

## Algorithms

In [ ]:
from myst_nb_bokeh import glue_bokeh

from gaitmap_bench import config, is_config_set
from gaitmap_bench.docu_utils import set_docs_config
from gaitmap_challenges.full_pipeline.kluge_2017 import Challenge
from gaitmap_challenges.results import load_run, get_latest_result, filter_results, get_all_result_paths, \
    generate_overview_table
from gaitmap_challenges.visualization import SingleMetricBoxplot, group_by_data_label

is_config_set() or set_docs_config()
all_runs = get_all_result_paths(Challenge, config().results_dir)
all_runs = filter_results(all_runs, challenge_version=Challenge.VERSION, is_debug_run=False)
latest_runs = get_latest_result(all_runs)
generate_overview_table(latest_runs).set_index("Entry").T

## Absolute Errors of the Mean

The main outcome are the absolute errors of the mean gait parameters.
This means one error value per gait test is calculated.
The first plot pools all datapoints across all 5 test folds.

The second plot presents the mean absolute error over each fold.

### Results per Participant and Test

In [ ]:
from gaitmap_bench.docu_utils import glue_bokeh_md, tabs
from myst_nb import glue
from IPython.display import Markdown

run_info = {k: load_run(Challenge, v) for k, v in latest_runs.items()}
cv_results = {k: v.results["cv_results"]  for k, v in run_info.items()}

tab_items = {}
metrics = {
    "Gait Velocity": "gait_velocity",
    "Stride Length": "stride_length",
    "Stride Time": "stride_time",
    "Swing Time": "swing_time",
}

for name, metric in metrics.items():
    p = SingleMetricBoxplot(cv_results, f"{metric}__abs_error", "single", overlay_scatter=True, label_grouper=group_by_data_label(level="speed", include_all="Combined", force_order=["slow", "normal", "fast", "Combined"]))
    glue_name = f"single_{metric}"
    glue_bokeh(glue_name, p.bokeh())
    tab_items[name] = glue_bokeh_md(glue_name)

glue("single_results", Markdown(tabs(tab_items, class_str="full-width")), display=False)

:::{glue:md} single_results
:format: myst
:::

### Results per CV Fold

In [ ]:
tab_items = {}

for name, metric in metrics.items():
    p = SingleMetricBoxplot(cv_results, f"agg__{metric}__abs_error_mean", "fold", overlay_scatter=True)
    glue_name = f"fold_{metric}"
    glue_bokeh(glue_name, p.bokeh())
    tab_items[name] = glue_bokeh_md(glue_name)

glue("fold_results", Markdown(tabs(tab_items, class_str="full-width")), display=False)

:::{glue:md} fold_results
:format: myst
:::

### Residual Plots

Below residual plots for each algorithm and metric are shown to further invesqtigate the dependency of the error on the actual measurement value.

In [ ]:
from gaitmap_challenges.visualization import ResidualPlot

final_markdown = []

for algo, results in cv_results.items():
    tab_items = {}
    for name, metric in metrics.items():
        p = ResidualPlot(results, prediction_col_name=f"{metric}__predicted", reference_col_name=f"{metric}__reference", metric_name=name)
        glue_name = f"residual_{algo}_{metric}"
        glue_bokeh(glue_name, p.bokeh())
        tab_items[name] = glue_bokeh_md(glue_name)
    final_markdown.append(f"#### {algo}")
    final_markdown.append(tabs(tab_items, class_str="full-width", sync=True))

glue("all_residual", Markdown("\n".join(final_markdown)), display=False)

:::{glue:md} all_residual
:format: myst
:::